# Cleaning Events data

### Importing necessary libraries

In [ ]:
pip install pandas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

### Loading the data

In [2]:
events = pd.read_csv('../sources/events.csv')
registrados = pd.read_csv('../sources/registered_in_events.csv')

## Data Cleaning

**Cleaning events**

**Cleaning registered_in_events**

**Joining both datasets**